In [ ]:
!pip install ../input/keras-applications/Keras_Applications-1.0.8/ -f ./ --no-index
!pip install ../input/image-classifiers/image_classifiers-1.0.0/ -f ./ --no-index
!pip install ../input/efficientnet-1-0-0/efficientnet-1.0.0/ -f ./ --no-index
!pip install ../input/segmentation-models/segmentation_models-1.0.1/ -f ./ --no-index

In [ ]:
%env SM_FRAMEWORK=tf.keras

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import cv2
import sys
import json
import time
import pickle
import shutil
import numba
import numpy as np
import pandas as pd 
import tifffile as tiff
import rasterio
from rasterio.windows import Window
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import *
import segmentation_models as sm
from segmentation_models import Unet, FPN
from segmentation_models.losses import bce_jaccard_loss
from tqdm import tqdm
print('tensorflow version:', tf.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    for gpu_device in gpu_devices:
        print('device available:', gpu_device)
pd.set_option('display.max_columns', None)

In [ ]:
DATA_PATH = '../input/hubmap-kidney-segmentation'
MDLS_PATH = '../input/hubmap-tf-with-tpu-inceptionresnet'

THRESHOLD = .275
VOTERS = 1
TTAS = [0, 1, 2, 3]
FOLDS = [0, 1, 2, 3, 4]
EXPAND = 2
MIN_OVERLAP = 300
IDNT = rasterio.Affine(1, 0, 0, 0, 1, 0)
STRATEGY = tf.distribute.get_strategy() 
SUB_PATH = f'{DATA_PATH}/test' 
TARGET_IMG = 'afa5e8098.tiff'
Y_SHFT = -40
X_SHFT = -24

start_time = time.time()

In [ ]:
import yaml
import pprint
with open(MDLS_PATH+'/params.yaml') as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
    pprint.pprint(params)

In [ ]:
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s) // 2):
            start = int(s[2 * i]) - 1
            length = int(s[2 * i + 1])
            img[start : start + length] = 1 + m
    return img.reshape(shape).T

def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def global_shift_mask(maskpred1, y_shift, x_shift):
    """
    applies a global shift to a mask by 
    padding one side and cropping from the other
    """
    if y_shift < 0 and x_shift >=0:
        maskpred2 = np.pad(maskpred1, 
                           [(0,abs(y_shift)), (abs(x_shift), 0)], 
                           mode='constant', constant_values=0)
        maskpred3 = maskpred2[abs(y_shift):, :maskpred1.shape[1]]
    elif y_shift >=0 and x_shift <0:
        maskpred2 = np.pad(maskpred1, 
                           [(abs(y_shift),0), (0, abs(x_shift))], 
                           mode='constant', constant_values=0)
        maskpred3 = maskpred2[:maskpred1.shape[0], abs(x_shift):]
    elif y_shift >=0 and x_shift >=0:
        maskpred2 = np.pad(maskpred1,
                           [(abs(y_shift),0), (abs(x_shift), 0)], 
                           mode='constant', constant_values=0)
        maskpred3 = maskpred2[:maskpred1.shape[0], :maskpred1.shape[1]]
    elif y_shift < 0 and x_shift < 0:
        maskpred2 = np.pad(maskpred1, 
                           [(0, abs(y_shift)), (0, abs(x_shift))], 
                           mode='constant', constant_values=0)
        maskpred3 = maskpred2[abs(y_shift):, abs(x_shift):]
    return maskpred3

In [ ]:
def make_grid(shape, window=256, min_overlap=32):
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx, ny, 4), dtype=np.int64) 
    for i in range(nx):
        for j in range(ny):
            slices[i, j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx * ny, 4)

def flip(img, axis=0):
    if axis == 1:
        return img[::-1, :, ]
    elif axis == 2:
        return img[:, ::-1, ]
    elif axis == 3:
        return img[::-1, ::-1, ]
    else:
        return img

In [ ]:
img_files = [x for x in os.listdir(SUB_PATH) if '.tiff' in x]
print('images idxs:', img_files)

In [ ]:
import glob
models_1 = []
for model_path in glob.glob(MDLS_PATH+'/*.h5'):
    models_1.append(tf.keras.models.load_model(model_path,compile = False))
print(len(models_1))

MDLS_PATH2 = '../input/efficient-net-b7'
models_2 = []
for model_path in glob.glob(MDLS_PATH2+'/*.h5'):
    models_2.append(tf.keras.models.load_model(model_path,compile = False))
print(len(models_2))

MDLS_PATH3 = '../input/efficientnetb4'
models_3 = []
for model_path in glob.glob(MDLS_PATH3+'/*.h5'):
    models_3.append(tf.keras.models.load_model(model_path,compile = False))
print(len(models_3))

MDLS_PATH4 = '../input/efficientnet-b6'
models_4 = []
for model_path in glob.glob(MDLS_PATH4+'/*.h5'):
    models_4.append(tf.keras.models.load_model(model_path,compile = False))
print(len(models_4))

In [ ]:
subm = {}
for i_img, img_file in enumerate(img_files):
    print('-' * 20, img_file, '-' * 20)
    img_data = rasterio.open(os.path.join(SUB_PATH, img_file), transform=IDNT)
    print('img shape:', img_data.shape)
    if img_data.count != 3:
        print('img file with subdatasets as channels')
        layers = [rasterio.open(subd) for subd in img_data.subdatasets]
    img_preds = np.zeros(img_data.shape, dtype=np.uint8)
    tile_size = int(params['DIM']*EXPAND)
    tile_resized = int(tile_size*params['DIM_FROM']/params['DIM'])
    slices = make_grid(
        img_data.shape, 
        window=tile_resized, 
        min_overlap=MIN_OVERLAP
    )
    
    for (x1, x2, y1, y2) in tqdm(slices, desc=f'{img_file}'):
        if img_data.count == 3: # normal
            img = img_data.read(
                [1, 2, 3], 
                window=Window.from_slices((x1, x2), (y1, y2))
            )
            img = np.moveaxis(img, 0, -1)
        else: # with subdatasets/layers
            img = np.zeros((tile_resized, tile_resized, 3), dtype=np.uint8)
            for fl in range(3):
                img[:, :, fl] = layers[fl].read(
                    window=Window.from_slices((x1, x2), (y1, y2))
                )
        img = cv2.resize(img, (tile_size, tile_size))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        pred = np.zeros((tile_size, tile_size), dtype=np.float32)
        for tta_mode in TTAS:
            img_aug = flip(img, axis=tta_mode)
            img_aug = np.expand_dims(img_aug, 0)
#             img_aug = img_aug.astype(np.float32) / 255
            pred_aug = np.zeros((tile_size, tile_size), dtype=np.float32)
            #for model1, model2, model3 in zip(models_1[:2], models_2[2:4], models_3[1:4]):
            pred_aug += 0.15*np.squeeze(models_1[0].predict(img_aug)) 
            pred_aug += 0.15*np.squeeze(models_1[4].predict(img_aug)) 
            pred_aug += 0.30*np.squeeze(models_2[1].predict(img_aug)) 
            pred_aug += 0.20*np.squeeze(models_3[2].predict(img_aug))
            pred_aug += 0.20*np.squeeze(models_4[3].predict(img_aug))
            pred += flip(pred_aug, axis=tta_mode) / len(TTAS)
        pred = cv2.resize(pred, (tile_resized, tile_resized))
        img_preds[x1:x2, y1:y2] = img_preds[x1:x2, y1:y2] + \
            (pred > THRESHOLD).astype(np.uint8)
    del img, pred, img_aug, pred_aug; gc.collect()
    print('img max:', np.max(img_preds), '| voters:', VOTERS)
    if img_file == TARGET_IMG:
        print('global shift')
        img_preds = (img_preds >= VOTERS).astype(np.uint8)
        img_preds = global_shift_mask(img_preds, Y_SHFT, X_SHFT)
    else:
        img_preds = (img_preds >= VOTERS).astype(np.uint8)
    rle_pred = rle_encode_less_memory(img_preds)
    subm[i_img] = {'id':img_file.replace('.tiff', ''), 'predicted': rle_pred}
    del img_preds, img_data, rle_pred; gc.collect()

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
df_sub = pd.DataFrame(subm).T
df_sub

In [ ]:
df_sub.to_csv('submission.csv', index=False)